In [86]:
import pandas as pd
import re
from datetime import datetime as dt

# Reading phrases

In [3]:
df = pd.read_csv('../../datasets/transcribations/transcribations_2023-07-01 00:00:00_2023-08-01 00:00:00.csv')

/tmp/ipykernel_56757/233176728.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../datasets/transcribations/transcribations_2023-07-01 00:00:00_2023-08-01 00:00:00.csv')


In [7]:
len(df)

5335316

In [15]:
df['text'] = df['text'].apply(lambda x: x.encode('ISO-8859-1').decode('windows-1251') if isinstance(x, str) else x)

In [16]:
df.head(2)

,Unnamed: 0,transcribation_date,date_y,date_m,date_d,side,text,start,audio_file_name,conf,...,ID,linkedid,dst,record_date,source_id,src,cpu_id,duration,file_size,queue_date
0,0,2023-07-01 06:14:36,NaN,NaN,NaN,True,что надо,32.16,in_5077_2023-07-01-06-02-56rxtx-out.wav,1.000000,...,133202961,1.688181e+09,main,2023-07-01 06:02:56,1,5077,1,39.9,638444,2023-07-01 06:14:22
1,1,2023-07-01 06:14:36,NaN,NaN,NaN,True,пошли вы нафиг вообще идиоты несчастные время ...,35.04,in_5077_2023-07-01-06-02-56rxtx-out.wav,0.998039,...,133202962,1.688181e+09,main,2023-07-01 06:02:56,1,5077,1,39.9,638444,2023-07-01 06:14:22


In [18]:
# Selecting only the specified columns
df = df[['text', 'side', 'linkedid', 'start']]
df.head(2)

,text,side,linkedid,start
0,что надо,True,1.688181e+09,32.16
1,пошли вы нафиг вообще идиоты несчастные время ...,True,1.688181e+09,35.04


In [19]:
df.to_csv('phrases07.csv')

### Read cat

In [26]:
# for idx, row in df.head(100).iterrows():
#     print(row.linkedid, row.start, row.side, row.text)

In [27]:
# Reloading the linkedid_cat.txt file by skipping the first row
linkedid_cat_path = 'linkedid_cat_07.txt'
linkedid_cat_df = pd.read_csv(linkedid_cat_path, delimiter='\t', skiprows=1, names=['linkedid', 'cat'])

# Displaying the first few rows of the linkedid_cat DataFrame
linkedid_cat_df.head(3)

,linkedid,cat
0,1688180574.7729737,NaN
1,1688180619.7729740,NaN
2,1688180649.7729742,NaN


In [28]:
linkedid_cat_df.dropna(inplace=True)
linkedid_cat_df.reset_index(drop=True, inplace=True)
linkedid_cat_df.head(3)

,linkedid,cat
0,1688184185.7729937,1.0
1,1688185557.7730424,1.0
2,1688185758.7730426,1.0


In [30]:
# Convert the 'linkedid' column to a string type to handle any NA / NaN values
linkedid_cat_df['linkedid'] = linkedid_cat_df['linkedid'].astype(str)
# Identify rows containing ".WAV" in the 'linkedid' column
rows_with_wav = linkedid_cat_df['linkedid'].str.endswith('.WAV')
# Remove ".WAV" from the 'linkedid' column
linkedid_cat_df.loc[rows_with_wav, 'linkedid'] = linkedid_cat_df.loc[rows_with_wav, 'linkedid'].str[:-4]

In [31]:
len(linkedid_cat_df)

35238

### Join phrases and cat

In [32]:
# Converting the 'linkedid' columns to float for proper join operation
df['linkedid'] = df['linkedid'].astype(float)
linkedid_cat_df['linkedid'] = linkedid_cat_df['linkedid'].astype(float)

/tmp/ipykernel_56757/284099278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['linkedid'] = df['linkedid'].astype(float)


In [35]:
# Joining the embeddings_df and linkedid_cat_df DataFrames on the 'linkedid' column
joined_df = pd.merge(df, linkedid_cat_df, on='linkedid', how='inner')

In [47]:
len(joined_df)

1265914

In [37]:
joined_df

,text,side,linkedid,start,cat
0,девочки пожалуйста меня бежит беспрерывно вода...,False,1.688186e+09,9.87,1.0
1,валентина александровна,False,1.688186e+09,23.40,1.0
2,да да да конечно,False,1.688186e+09,28.29,1.0
3,да мне прям сейчас вот пускай приезжает сейчас,False,1.688186e+09,38.70,1.0
4,ой вы мне присылаете я очень давно пользуюсь в...,False,1.688186e+09,46.26,1.0
...,...,...,...,...,...
1271129,девяносто один,False,1.690833e+09,88.17,1.0
1271130,семьдесят девять,False,1.690833e+09,90.24,1.0
1271131,да да да,False,1.690833e+09,95.91,1.0
1271132,ну да да да,False,1.690833e+09,101.73,1.0


In [39]:
categories = {
    0:'-',
    1:'Заявка',
    2:'Заявка не создана',
    3:'Повторный звонок по заявке',
    4:'Доп. услуги нашей компании',
    5:'Некоммерческая деятельность, ошибки'
} 

In [46]:
# Drop rows where the 'text' column is NaN, in place
joined_df.dropna(subset=['text'], inplace=True)

In [51]:
# Sort the DataFrame by 'linkedid' and 'start' columns, in place
joined_df.sort_values(by=['linkedid', 'start'], inplace=True)

In [57]:
joined_df.to_csv('joined_df.csv')

### Split operators and customers

In [65]:
operators_df = joined_df[joined_df.side==True]
len(operators_df)

604700

In [60]:
# for idx, row in joined_df[joined_df['side']==True][:100].iterrows():
#     print(row.linkedid, row['cat'], row.start, row.side, row.text)

In [66]:
operators_df.head(2)

,text,side,linkedid,start,cat
26,центра пётр наталья чем могу помочь,True,1.688186e+09,6.33,1.0
27,скажите пожалуйста как я могу к вам обращаться,True,1.688186e+09,20.49,1.0


### Drop extra categories

In [67]:
# Drop rows where 'cat' is 1 or 2, in place
operators_df = operators_df[operators_df['cat'].isin([1, 2])]

In [68]:
len(operators_df)

391003

In [72]:
# Set 'cat' to 0 where 'cat' is 2
operators_df.loc[operators_df['cat'] == 2, 'cat'] = 0

In [77]:
len(operators_df[operators_df['cat']==0])

24520

In [78]:
len(operators_df[operators_df['cat']==1])

366483

In [96]:
operators_df.to_csv('operators_df.csv')

# Embeddings

In [97]:
operators_df = pd.read_csv('operators_df.csv')

In [93]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel

In [94]:
# Define the device (use GPU if available, otherwise use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Loading model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# Move the model to the specified device
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-5): 6 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [88]:
def get_sentence_embeddings(df):
    # Sorting values
    # top_n = 1000
    df = df.sort_values("linkedid")
    # df = df.sort_values("linkedid").tail(top_n * 2)    

    # Ensure that all entries in 'text' column are strings
    sentences = df['text'].astype(str).tolist()

    # Create DataLoader for batching
    batch_size = 32  # Adjust based on your GPU's memory
    dataloader = DataLoader(sentences, batch_size=batch_size, shuffle=False)

    embeddings_list = []
    
    for batch in dataloader:
        # Tokenize sentences
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

        # Move the encoded input to the specified device
        encoded_input = encoded_input.to(device)

        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input)

        # Apply mean pooling to get sentence embeddings
        attention_mask = encoded_input['attention_mask']
        embeddings = mean_pooling(model_output, attention_mask).cpu().numpy()
        embeddings_list.extend(embeddings)

    # Add embeddings to the DataFrame
    df['embedding'] = embeddings_list
    # df = df.tail(top_n)  # Keep only the top_n entries

    return df

# Mean Pooling function
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [95]:
torch.cuda.is_available()

False

In [90]:
device

device(type='cpu')

In [89]:
print(dt.now())
df = get_sentence_embeddings(df)
print(dt.now())
df.head(3)

2023-08-25 08:59:21.073180


KeyboardInterrupt: 